# Striking a Fine Balance Between Fast and Easy, and Fast and Accurate
> schema-enforcement is policiing the precious 'structure' of your 'existing' table.
> if it does the job right, then no harm may come to said 'schema'

## Use Case One: Import to Delta from Parquet to generate initial Schema

In [57]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col, concat_ws, sha1
from typing import Dict, Optional, Union

In [2]:
table_path = "/opt/spark/work-dir/hitchhikers_guide/datasets/ecomm_behavior_data/delta/ecomm/"

# using an existing table (we must not screw this up!)
dt = DeltaTable.forPath(spark, table_path)

In [3]:
results = (
    dt.toDF()
    .where(
        col("event_date").isin(["2019-10-01","2019-10-02"])
    )
    .limit(10)
)

In [ ]:
results.show()

In [13]:
(
    dt.toDF()
    .alias("ecomm_orig")
    .select(
        "ecomm_orig.event_time", 
        "ecomm_orig.product_id", 
        "ecomm_orig.category_id")
    .show()
)

+-------------------+----------+-------------------+
|         event_time|product_id|        category_id|
+-------------------+----------+-------------------+
|2019-10-25 07:27:49|  28600035|2053013558282682943|
|2019-10-25 00:00:00|  12100514|2053013555816432043|
|2019-10-25 07:27:49|   1005098|2053013555631882655|
|2019-10-25 00:00:00|  15100009|2053013557024391671|
|2019-10-25 07:27:49|  26204088|2053013563693335403|
|2019-10-25 00:00:00|   1004723|2053013555631882655|
|2019-10-25 07:27:49|   1201292|2172371436436455782|
|2019-10-25 00:00:00|   1005255|2053013555631882655|
|2019-10-25 07:27:49|   3600164|2053013563810775923|
|2019-10-25 00:00:00|  13201079|2053013557192163841|
|2019-10-25 07:27:50|   1004788|2053013555631882655|
|2019-10-25 00:00:01|   1005157|2053013555631882655|
|2019-10-25 07:27:50|   4804295|2053013554658804075|
|2019-10-25 00:00:01|   1201361|2172371436436455782|
|2019-10-25 07:27:50|   3200321|2053013555321504139|
|2019-10-25 00:00:01|  10400081|20530135532579

## Schema Enforcement is Here for Us. It isn't acting against us.
> It is normal for us to need to make changes to our tables.
> 
> After all, that is part of our job description. The way we make the modifications is what is really important.

Ask yourself if you are making the _correct_ change to a given table. Also ask yourself if you are making a _breaking_change_?



In [68]:
# If we add a random column to our `results` 
# and then overwrite the two days of data using the `event_time` column in our `partitionBy` clause — that could corrupt the rest of the table (if this was parquet)

# this is simple enough: concat two columns, and then hash them with sha1, and add a new column named `category_id_event_type_col` (bad name... I know!)
lets_break_stuff = (
    results.withColumn(
        "category_id_event_type_col", 
        sha1(concat_ws("_", col("event_type"), col("category_id")))
    )
)

In [69]:
lets_break_stuff.show()

+-------------------+----------+----------+-------------------+--------------------+----------+-------+---------+--------------------+----------+--------------------------+
|         event_time|event_type|product_id|        category_id|       category_code|     brand|  price|  user_id|        user_session|event_date|category_id_event_type_col|
+-------------------+----------+----------+-------------------+--------------------+----------+-------+---------+--------------------+----------+--------------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                NULL|  shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|2019-10-01|      04fea625d663b0f33...|
|2019-10-01 10:45:06|      view|  28717654|2053013565639492569|       apparel.shoes|      NULL|   99.1|548846299|2cef6570-ab23-4d2...|2019-10-01|      5b9e1f86433535cc5...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|      aqua|   33.2|554748717|9333dfbd-b87a-470...|2

## Delta Will Step in to Stop Us if we are being Naughty
> Go ahead. Try to modify the schema.

The following will attempt to add the new `category_id_event_type_col` and drop the useful `product_id` column. While this is just an example of how Delta Schema Enforcement comes to the rescue and saves the day, it is also something to be aware of. We don't need to break things by accident. 

In [70]:
# now here is where we would be making a potentially breaking change
# if we drop `product_id` then we no longer understand what "was purchased" and the ecomm data no longer is valueable (at least to me).
(
    lets_break_stuff
    .drop("product_id")
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("event_date")
    .option("path", table_path)
    .save()
)

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: e09582e2-ff59-47ba-a624-ff6d6855dcb5).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- event_time: timestamp (nullable = true)
-- event_type: string (nullable = true)
-- product_id: integer (nullable = true)
-- category_id: long (nullable = true)
-- category_code: string (nullable = true)
-- brand: string (nullable = true)
-- price: float (nullable = true)
-- user_id: integer (nullable = true)
-- user_session: string (nullable = true)
-- event_date: date (nullable = true)


Data schema:
root
-- event_time: timestamp (nullable = true)
-- event_type: string (nullable = true)
-- category_id: long (nullable = true)
-- category_code: string (nul

# We have Options when it comes to How We Modify the Table
> Just like in life. There are always choices to make. For Delta, we can either "automatically" adapt to schema changes (if we trust our upstream data sources), or we can "opt-in" to schema changes.

## When to Automatically Opt-In to Schema Changes
If you have a high level of trust in the upstream data product (source, team, owners) and you want to "allow" changes to flow that respect **type saftey** then we can use `.option("mergeSchema", "true")` to automatically "reflect" changes.

## When to Opt-In to Schema Changes
If we ignore the fact that we can **automatically** merge type-safe changes to our Delta tables, then we can run traditional `ALTER TABLE name ADD COLUMNS` to manually reflect the changes to the table we are expecting. This is the traditional route, but when it comes to "what" flows downstream. Always remember that it is better to have a job fail with an AnalysisException than for potentially problematic changes to blindly flow down to your own data customers.


In [71]:
# Opt-in to the correct changes
# if we know we want to keep the change. Then let's do this with respect
spark.sql(f"""
ALTER TABLE delta.`{table_path}`
ADD COLUMNS (
  category_id_event_type_col STRING
)
""")

DataFrame[]

In [25]:
dt2 = DeltaTable.forPath(spark, table_path)
dt2.history(1).select("version", "operation", "operationParameters").show(truncate=False)

+-------+---------+--------------------------------------------------+
|version|operation|operationParameters                               |
+-------+---------+--------------------------------------------------+
|19     |WRITE    |{mode -> Overwrite, partitionBy -> ["event_date"]}|
+-------+---------+--------------------------------------------------+



In [78]:
# Use Automatic Merges with a Grain of Salt
# The above could have been done by changing the query from 

(
    lets_break_stuff
    .drop("product_id")
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("event_date")
    .option("mergeSchema", "true")
    .option("path", table_path)
    .save()
)

In [79]:
dt2.history(1).select("version", "operation", "operationParameters").show(truncate=False)

+-------+---------+--------------------------------------------------+
|version|operation|operationParameters                               |
+-------+---------+--------------------------------------------------+
|15     |WRITE    |{mode -> Overwrite, partitionBy -> ["event_date"]}|
+-------+---------+--------------------------------------------------+



In [83]:
dt2.toDF().printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: float (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- event_date: date (nullable = true)
 |-- category_id_event_type_col: string (nullable = true)



## It is worth pointing out that "Delta" mergeSchema is "additive"
> In the above example, we `dropped` the `product_id` column, but Delta doesn't care - this will just be a null column
> This is a stupid thing to have done, since this means the data now has `null` values vs having `real` values!

Luckily, even if we make mistakes we can roll them back.

In [85]:
dt2.toDF().where(col("event_date").isin(["2019-10-01","2019-10-02"])).show()

+-------------------+----------+----------+-------------------+--------------------+----------+-------+---------+--------------------+----------+--------------------------+
|         event_time|event_type|product_id|        category_id|       category_code|     brand|  price|  user_id|        user_session|event_date|category_id_event_type_col|
+-------------------+----------+----------+-------------------+--------------------+----------+-------+---------+--------------------+----------+--------------------------+
|2019-10-01 00:00:00|      view|      NULL|2103807459595387724|                NULL|  shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|2019-10-01|      04fea625d663b0f33...|
|2019-10-01 10:45:06|      view|      NULL|2053013565639492569|       apparel.shoes|      NULL|   99.1|548846299|2cef6570-ab23-4d2...|2019-10-01|      5b9e1f86433535cc5...|
|2019-10-01 00:00:00|      view|      NULL|2053013552326770905|appliances.enviro...|      aqua|   33.2|554748717|9333dfbd-b87a-470...|2

# Can you spot the NULL mistake
> We dropped the `product_id` column. Not great. Let's get it back

In [8]:
dt2.history(10).show()

+-------+--------------------+------+--------+-----------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|  operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+-----------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|     16|2024-03-21 00:12:...|  NULL|    NULL|    RESTORE|{version -> 14, t...|NULL|    NULL|     NULL|         15|  Serializable|        false|{numRestoredFiles...|        NULL|Apache-Spark/3.5....|
|     15|2024-03-21 00:06:...|  NULL|    NULL|      WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|         14|  Serializable|        false|{numFiles -> 1, n...|        NULL|Apache-Spark/3.5....|


In [ ]:
dt2.restoreToVersion(14)

In [ ]:
dt2.toDF().where(col("event_date").isin(["2019-10-01","2019-10-02"])).show()

## Great. Even when Schema Enforcement works but we "do naughty things" we can restore back to where we were.
> No harm. No Foul (or fowl if we are birds...)

# Overwriting a Table Schema.
> Only Do this if you really need to make drastic changes.
> This will reorganize the underlying directory structure of a given table
> This will cause all downstream "streaming" jobs to fail. Can cause cascading problems if not communicated well.

In [ ]:
(
  dt2
  .toDF()
  .drop("category_id_event_type_col")
  .write
  .format("delta")
  .mode("overwrite")
  .partitionBy("event_date")
  .option("overwriteSchema", "true")
  .option("path", table_path)
  .save()
)

In [23]:
dt2.history(1).select("version", "operation", "operationParameters").show(truncate=False)

+-------+---------+--------------------------------------------------+
|version|operation|operationParameters                               |
+-------+---------+--------------------------------------------------+
|19     |WRITE    |{mode -> Overwrite, partitionBy -> ["event_date"]}|
+-------+---------+--------------------------------------------------+



# If you actually liked the last version....
> these numbers will potentially be different. Just use the `dt2.history()` to find the version of the table you want to restore to.

In [ ]:
dt2.restoreToVersion(18)

In [26]:
dt2.toDF().printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: float (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- event_date: date (nullable = true)

